# Routes with Genetic Algorithm

### Created by:
 - Héctor Aguirre
 - Carlos García

In [ ]:
from functools import cmp_to_key
from math import floor
import numpy as np
from geneticalgorithm import geneticalgorithm as ga

In [ ]:
def compare(X1, X2):
    if X1[0] < X2[0]:
        return -1
    elif X1[0] > X2[0]:
        return 1
    elif X1[1] < X2[1]:
        return -1
    elif X1[1] > X2[1]:
        return 1
    else:
        return 0

def dist(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

### The function runGenetic gets 2 parameters:
- Beginning
    - Its a point in a 3D plane where all the routes will start
    - Its an array with 3 elements [x, y, 0] where those are the coords X,Y,Z 

- Destination
    - Its the array of n points (array with 3 elements just like beginning).
    - An example: [[x, y, 0], [x, y, 0], [x, y, 0], [x, y, 0]]

In [ ]:
#count = 0
def runGenetic(beginning, destinations):
    dim=len(destinations)

    def f(X):
        global count
        #count +=1
        carCount = X[0]
        exes = []
        penalty = 10000000
        maxCarIndex = -1
        for i in range(1, len(X), 2):
            if X[i] >= carCount:
                return penalty
            maxCarIndex = max(maxCarIndex, X[i])
            exes.append([X[i], X[i+1]])
        
        if carCount != maxCarIndex+1:
            return penalty
        exes.sort(key=cmp_to_key(compare))

        if exes[0][0] != 0 or exes[0][1] != 0:
            return penalty
        
        for i in range(len(exes)-1):
            if not(exes[i+1][0] == exes[i][0] and exes[i+1][1] == exes[i][1]+1) and not(exes[i+1][0] == exes[i][0]+1 and exes[i+1][1] == 0):
                return penalty

        accum = 0
        for i in range(1, len(X), 2):
            destIndex = floor((i-1)/2)
            carIndex = X[i]
            orderInCar = X[i+1]
            if orderInCar == 0:
                accum += dist(destinations[destIndex], beginning)
            else:
                for j in range(1, len(X), 2):
                    if X[j] == carIndex and X[j+1] == orderInCar-1:
                        accum += dist(destinations[destIndex], destinations[floor((j-1)/2)])
        return accum

    varbound=np.array([[1, dim]] + [[0, dim-1]]*(dim*2))

    model=ga(function=f,dimension=dim*2+1,variable_type='int',variable_boundaries=varbound, convergence_curve=False)

    model.run()
    #print('count ', count)
    return model.output_dict['variable']


## Model shape

### First element: 
#### - number of cars we will use
### Every following pair: 
#### - First element: Car index that will have that destination 
#### - Second element: Car order in the route

In [ ]:
from tkinter import *
from math import floor
from PIL import Image
from PIL import ImageTk
import random

# GUI made with Tkinter library

Initializate the app like window size, point size, etc.


In [ ]:
canvas_width = 800
canvas_height = 800
point_size = 5
beginning_point_size = 7

destinations = []

button = None
solved = False


## Functions

### Paint
This function decides whether to paint a clicked point as the beggining point or as a destination point.

### addBeginning
This function paints the beggining point with a purple dot on the location of the click.

### addDestination
This function paints a new destination point with a black dot on the location of the click.

### connectDestinations
This function draws a line between points of the same generated route

### generate
This function runs the Genetic Algorithm (runGenetic function) and saves the best result  


In [ ]:
def paint(event):
    if not(solved):
        if 'beginning' in globals():
            addDestination(event)
        else:
            addBeginning(event)
    

def addBeginning(event):
    global beginning
    beginning = [event.x, event.y, 0]
    purple = "#AA336A"
    x1, y1 = (event.x - beginning_point_size), (event.y - beginning_point_size)
    x2, y2 = (event.x + beginning_point_size), (event.y + beginning_point_size)
    w.create_oval(x1, y1, x2, y2, fill=purple, outline='')

def addDestination(event):
    destinations.append([event.x, event.y, 0])
    black = "#000000"
    x1, y1 = (event.x - point_size), (event.y - point_size)
    x2, y2 = (event.x + point_size), (event.y + point_size)
    w.create_oval(x1, y1, x2, y2, fill=black, outline='')

def connectDestinations(dest1, dest2, car, carColors):
    if not (car in carColors):
        r = lambda: random.randint(0,255)
        carColors[car] = '#%02X%02X%02X' % (r(),r(),r())
    w.create_line(dest1[0], dest1[1], dest2[0], dest2[1], width=3, fill=carColors[car] )

def generate():
    carColors = {}
    solution = runGenetic(beginning, destinations)
    print("------------------------",solution)
    for i in range(1, len(solution), 2):
        destIndex1 = floor((i-1)/2)
        if solution[i+1] == 0:
            #trace line from destinations[destIndex1] to beginning
            connectDestinations(destinations[destIndex1], beginning, solution[i], carColors)
        else:
            for j in range(1, len(solution), 2):
                destIndex2 = floor((j-1)/2)
                if solution[i] == solution[j] and solution[i+1] -1 == solution[j+1]:
                    #trace line between destinations[destIndex1] and destinations[destIndex2]
                    connectDestinations(destinations[destIndex1], destinations[destIndex2], solution[i], carColors)
    if button:
        button['state'] = 'disabled'
    global solved
    solved = True

### Instance of Tkinter GUI

In [ ]:
master = Tk()
master.title("Points")
w = Canvas(master,
           width=canvas_width,
           height=canvas_height)
w.pack(expand=YES, fill=BOTH)
w.bind("<Button-1>", paint)

img = Image.open("mapa.png")
img = img.resize((canvas_width,canvas_height), Image.ANTIALIAS)
photoImg =  ImageTk.PhotoImage(img)
image = w.create_image(canvas_width, 0, anchor=NE, image=photoImg)
button = Button(master, text='Generate routes', width=25, command=generate)
button.pack()
mainloop()